In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 3
setup_one_gpu(GPU)

Picking GPU 3


In [2]:
import csv
import pandas
import numpy as np
import tensorflow as tf
import pandas as pd
import os.path as osp
import matplotlib.pylab as plt
from pandas import DataFrame
from general_tools.in_out.basics import files_in_subdirs
from general_tools.in_out.basics import pickle_data, unpickle_data
from tf_lab.point_clouds.encoders_decoders import decoder_with_fc_only
from tf_lab.point_clouds.in_out import PointCloudDataSet, train_validate_test_split

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [4]:
close_map_file = '/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/3d_neighbs_as_context/pilot_triplets/outlier_frac_30/closest_nn/data/model_names_of_triplets.txt'
split_map_file = '/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/3d_neighbs_as_context/pilot_triplets/outlier_frac_30/one_far/data/model_names_of_triplets.txt'
far_map_file = '/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/3d_neighbs_as_context/pilot_triplets/outlier_frac_30/both_far/data/model_names_of_triplets.txt'

In [3]:
def read_triplets(triplets_file):
    res = []
    with open(triplets_file, 'r') as fin:
        for line in fin:
            tokens = line.split()
            res.append([tokens[1], tokens[2], tokens[3]])
        return res

In [109]:
triplet_ids = read_triplets(close_map_file)
A = [i[0] for i in triplet_ids]
B = [i[1] for i in triplet_ids]
C = [i[2] for i in triplet_ids]
# lala = set(np.unique(B))
# lele = set(np.unique(C))
# lala.intersection(lele)

In [47]:
def process_data_of_single_condition(triplets_file, geo_features):
    triplet_ids = read_triplets(triplets_file)
    selected_feats = np.zeros(shape=(len(triplet_ids), geo_features.shape[1] * 3))
    labels = np.zeros(shape=(len(triplet_ids), 3), dtype=np.int32)
    
    for i, tr in enumerate(triplet_ids):
        geo_f = geo_features.loc[tr].as_matrix()
        index = np.arange(3)
        np.random.shuffle(index)
        geo_f = geo_f[index, :]
        selected_feats[i] = np.hstack(geo_f)        
        new_pos = np.where(index==0)[0][0]  # target is expexted to be first vector.
        labels[i, new_pos] = 1 # one hot embedding
    return selected_feats, labels

In [27]:
geo_embedding = pandas.read_pickle('/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/3d_neighbs_as_context/pilot_chair_embeddings.pkl')

In [48]:
close_feats, close_labels = process_data_of_single_condition(close_map_file, geo_embedding)
far_feats, far_labels = process_data_of_single_condition(far_map_file, geo_embedding)
split_feats, split_labels = process_data_of_single_condition(split_map_file, geo_embedding)

# all_cond_feats = np.vstack([close_feats, far_feats, split_feats])
# all_cond_labels = np.vstack([close_labels, far_labels, split_labels])

all_cond_feats = close_feats
all_cond_labels = close_labels

In [50]:
train_data, _, test_data, _ = train_validate_test_split([all_cond_feats, all_cond_labels, np.arange(len(all_cond_labels))], train_perc=0.8, validate_perc=0, test_perc=0.2, shuffle=True)
train_data = PointCloudDataSet(point_clouds=train_data[0], labels=train_data[1], noise=train_data[2])
test_data = PointCloudDataSet(point_clouds=test_data[0], labels=test_data[1], noise=test_data[2])

In [105]:
n_classes = 3
geo_feat_size = 128 * n_classes
learning_rate = 0.003
train_epochs = 20
batch_size = 40

In [106]:
input_geometry = tf.placeholder(tf.float32, [None, geo_feat_size])
target_label = tf.placeholder(tf.int32, [None, n_classes])

In [107]:
logits = decoder_with_fc_only(input_geometry, layer_sizes=[100, 200, n_classes])
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, target_label)
cross_entropy = tf.reduce_mean(cross_entropy)

prediction = tf.argmax(logits, axis=1)
correct_pred = tf.equal(prediction, tf.argmax(target_label, axis=1))
avg_accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

opt_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

In [108]:
gpu_config = tf.ConfigProto()
gpu_config.gpu_options.allow_growth = True
sess = tf.Session(config=gpu_config)

In [110]:
init = tf.global_variables_initializer()
sess.run(init)    
for epoch in range(20):
    for _ in range(train_data.num_examples/batch_size):
        batch_geo, batch_labels, _ = train_data.next_batch(batch_size)
        feed_dict = {input_geometry: batch_geo, target_label: batch_labels}
        sess.run([opt_step], feed_dict=feed_dict)
    if epoch % 5 == 0:
        print sess.run([cross_entropy, avg_accuracy], feed_dict=feed_dict)        

[1.041082, 0.5]
[0.93317997, 0.57500005]
[0.55108976, 0.80000001]
[0.32128039, 0.94999999]


In [111]:
# TEST DATA        
batch_geo, batch_labels, _ = test_data.next_batch(test_data.num_examples)
feed_dict = {input_geometry: batch_geo, target_label: batch_labels}
print sess.run([avg_accuracy], feed_dict=feed_dict)

[0.2857143]


In [103]:
a = set(test_data.noisy_point_clouds)
a.intersection(train_data.noisy_point_clouds)


set()

# Conclusion
I have deciphered the issue of the classification.
There is no bias - or any problem that you have to worry about the data that I gave to Judy. (edited)
In fact I got more intuition on the matter.
Here is the two cents:  when we give to a NN only the latent geometric codes
it can infer in the test data the “target” with ~33% when the training/test data come only from the  close condition.
I.e., it does random guessing in the test data set.
But when we train/test with the split/far condition respectively, it guesses 50%/60%  correctly in the test data.
This is because it is clever enough to pick up the correlations implied by the way I construct the “far-split” conditions. I.e., in the far: both distractors are always “far” from the target, but we don’t constraint the between distance -> thus we give a clue to the network about who is the target. (edited)
Similarly, we do for the “split”.